In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import VotingClassifier
from imblearn.under_sampling import *
from imblearn.combine import SMOTETomek
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA # 차원축소 
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import MinMaxScaler# 성능이 좋은 변수만 사용하는 전처리기
import warnings
from sklearn.preprocessing import StandardScaler


warnings.filterwarnings('ignore')

In [4]:
human = pd.read_csv('human.csv', encoding = 'cp949')
human.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,성별,자본 이득,자본 손실,주당 시간,모국
0,H20001,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,H20002,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,H20003,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,H20004,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,H20005,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [5]:
human['성별'].value_counts()

 Male      21790
 Female    10771
Name: 성별, dtype: int64

In [6]:
#타입 바꾸기
obj = ['성별','노동 계급','학력','혼인 상태','직업','관계','인종','모국']
human[obj] = human[obj].apply(lambda x: x.astype('category').cat.codes)

In [7]:
human.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,성별,자본 이득,자본 손실,주당 시간,모국
0,H20001,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38
1,H20002,50,5,83311,9,13,2,3,0,4,1,0,0,13,38
2,H20003,38,3,215646,11,9,0,5,1,4,1,0,0,40,38
3,H20004,53,3,234721,1,7,2,5,0,2,1,0,0,40,38
4,H20005,28,3,338409,9,13,2,9,5,2,0,0,0,40,4


In [8]:
human = human.rename(columns={'아이디':'id', '나이':'age', '노동 계급':'workclass', '학력':'education', 
              '교육 수':'education_num', '혼인 상태':'marital_status', '직업':'occupation',
              '관계':'relationship', '인종':'race', '성별':'sex', '자본 이득':'capital_gain',
              '자본 손실':'capital_loss', '주당 시간':'hours_per_week', '모국':'native_country'})
human

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,H20001,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38
1,H20002,50,5,83311,9,13,2,3,0,4,1,0,0,13,38
2,H20003,38,3,215646,11,9,0,5,1,4,1,0,0,40,38
3,H20004,53,3,234721,1,7,2,5,0,2,1,0,0,40,38
4,H20005,28,3,338409,9,13,2,9,5,2,0,0,0,40,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,H52557,27,3,257302,7,12,2,12,5,4,0,0,0,38,38
32557,H52558,40,3,154374,11,9,2,6,0,4,1,0,0,40,38
32558,H52559,58,3,151910,11,9,6,0,4,4,0,0,0,40,38
32559,H52560,22,3,201490,11,9,4,0,3,4,1,0,0,20,38


In [9]:
human=human.fillna('*')

In [10]:
human.corr()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
age,1.000000,0.003787,-0.076646,-0.010508,0.036527,-0.266288,-0.020947,-0.263698,0.028718,0.088832,0.077674,0.057775,0.068756,-0.001151
workclass,0.003787,1.000000,-0.016656,0.023513,0.052085,-0.064731,0.254892,-0.090461,0.049742,0.095981,0.033835,0.012216,0.138962,-0.007690
fnlwgt,-0.076646,-0.016656,1.000000,-0.028145,-0.043195,0.028153,0.001597,0.008931,-0.021291,0.026858,0.000432,-0.010252,-0.018768,-0.051966
education,-0.010508,0.023513,-0.028145,1.000000,0.359153,-0.038407,-0.021260,-0.010876,0.014131,-0.027356,0.030046,0.016746,0.055510,0.064288
education_num,0.036527,0.052085,-0.043195,0.359153,1.000000,-0.069304,0.109697,-0.094153,0.031838,0.012280,0.122630,0.079923,0.148123,0.050840
marital_status,-0.266288,-0.064731,0.028153,-0.038407,-0.069304,1.000000,-0.009654,0.185451,-0.068013,-0.129314,-0.043393,-0.034187,-0.190519,-0.023819
occupation,-0.020947,0.254892,0.001597,-0.021260,0.109697,-0.009654,1.000000,-0.075607,0.006763,0.080296,0.025505,0.017987,0.080383,-0.012543
relationship,-0.263698,-0.090461,0.008931,-0.010876,-0.094153,0.185451,-0.075607,1.000000,-0.116055,-0.582454,-0.057919,-0.061062,-0.248974,-0.005507
race,0.028718,0.049742,-0.021291,0.014131,0.031838,-0.068013,0.006763,-0.116055,1.000000,0.087204,0.011145,0.018899,0.041910,0.137852
sex,0.088832,0.095981,0.026858,-0.027356,0.012280,-0.129314,0.080296,-0.582454,0.087204,1.000000,0.048480,0.045567,0.229309,-0.008119


In [11]:
from sklearn.model_selection import train_test_split

x = human.drop(['id','sex'], axis=1)
y = human['sex']
#y = train.GENDER.values
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(24420, 13) (8141, 13) (24420,) (8141,)


- SMOTE + Tomek 
    - 복합샘플링(Combining Over-and Under-Sampling), Hybrid 샘플링

In [16]:
# Tomek links: a under-sampling class method
XX, yy = SMOTETomek().fit_sample(X_train, y_train)
tree0 =  GradientBoostingClassifier(n_estimators=200, random_state=0)
tree0.fit(XX, yy)
y_pred0 = tree0.predict(X_test)

print(classification_report(y_test, y_pred0))

              precision    recall  f1-score   support

           0       0.70      0.85      0.77      2701
           1       0.92      0.82      0.87      5440

    accuracy                           0.83      8141
   macro avg       0.81      0.84      0.82      8141
weighted avg       0.85      0.83      0.83      8141



In [17]:
XX, yy = SMOTETomek(random_state=0).fit_sample(X_train, y_train)

In [18]:
tree3 =  GradientBoostingClassifier(n_estimators=200, random_state=0)
tree3.fit(XX, yy)
y_pred3 = tree3.predict(X_test)

print(classification_report(y_test, y_pred3))
# (precision) 0 이라고 예측한 데이터의 74%만 실제로 0
# (precision) 1 이라고 예측한 데이터의 33%만 실제로 1
# (recall) 실제 0인 데이터중 56%만 0으로 판별됨 
# (recall) 실제 1인 데이타중 52%만 1로 판별됨

              precision    recall  f1-score   support

           0       0.70      0.84      0.76      2701
           1       0.91      0.82      0.86      5440

    accuracy                           0.83      8141
   macro avg       0.81      0.83      0.81      8141
weighted avg       0.84      0.83      0.83      8141



In [19]:
tree3.score(X_test,y_test) #샘플링 안한게 더 나음

0.8283994595258568

##### 1. Decision Trees #####

In [101]:
tree = DecisionTreeClassifier(max_depth=6, random_state=0)

In [102]:
tree.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [103]:
pred_tree = tree.predict(X_test); pred_tree

array([0, 0, 1, ..., 1, 1, 0], dtype=int8)

In [104]:
display(tree.score(X_test, y_test))

0.825451418744626

In [106]:
humanew =pd.read_csv('human_new.csv',encoding='cp949')
humanew.head()

,아이디,나이,노동 계급,fnlwgt,학력,교육 수,혼인 상태,직업,관계,인종,자본 이득,자본 손실,주당 시간,모국
0,H0001,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,0,0,40,United-States
1,H0002,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,0,0,50,United-States
2,H0003,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,0,0,40,United-States
3,H0004,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,7688,0,40,United-States
4,H0005,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,0,0,30,United-States


In [107]:
#타입 바꾸기
obj = ['노동 계급','학력','혼인 상태','직업','관계','인종','모국']
humanew[obj] = humanew[obj].apply(lambda x: x.astype('category').cat.codes)

In [108]:
humanew = humanew.rename(columns={'아이디':'id', '나이':'age', '노동 계급':'workclass', '학력':'education', 
              '교육 수':'education_num', '혼인 상태':'marital_status', '직업':'occupation',
              '관계':'relationship', '인종':'race', '자본 이득':'capital_gain',
              '자본 손실':'capital_loss', '주당 시간':'hours_per_week', '모국':'native_country'})
humanew

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,capital_gain,capital_loss,hours_per_week,native_country
0,H0001,25,3,226802,1,7,4,6,3,2,0,0,40,37
1,H0002,38,3,89814,11,9,2,4,0,4,0,0,50,37
2,H0003,28,1,336951,7,12,2,10,0,4,0,0,40,37
3,H0004,44,3,160323,15,10,2,6,0,2,7688,0,40,37
4,H0005,18,-1,103497,15,10,4,-1,3,4,0,0,30,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,H16277,39,3,215419,9,13,0,9,1,4,0,0,36,37
16277,H16278,64,-1,321403,11,9,6,-1,2,2,0,0,40,37
16278,H16279,38,3,374983,9,13,2,9,0,4,0,0,50,37
16279,H16280,44,3,83891,9,13,0,0,3,1,5455,0,40,37


In [109]:
humanew['SEX'] = tree.predict(humanew.loc[:,'age':'native_country'])

##### 2. SVM

In [110]:
from sklearn.svm import SVC 
svm = SVC(random_state=0)
svm.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [111]:
display(svm.score(X_test, y_test))

0.6680997420464316

##### 3. Neural Networks

In [112]:
from sklearn.neural_network import MLPClassifier 
mlp = MLPClassifier()
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [113]:
display(mlp.score(X_test, y_test))

0.5263481144822504

In [114]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
pred_dummy = dummy.predict(X_test)
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
pred_tree = tree.predict(X_test)

In [115]:
from sklearn.metrics import accuracy_score
print("Dummy model:")
print(accuracy_score(y_test, pred_dummy))
print("Decision tree:")
print(accuracy_score(y_test, pred_tree))

Dummy model:
0.6682225770789829
Decision tree:
0.7750890553985997


In [116]:
from sklearn.metrics import confusion_matrix
print("Dummy model:")
print(confusion_matrix(y_test, pred_dummy))
print("Decision tree:")
print(confusion_matrix(y_test, pred_tree))

Dummy model:
[[   0 2701]
 [   0 5440]]
Decision tree:
[[1066 1635]
 [ 196 5244]]


In [117]:
logreg = LogisticRegression()
tree = DecisionTreeClassifier()
knn = KNeighborsClassifier()
voting = VotingClassifier(
    estimators = [('logreg', logreg), ('tree', tree), ('knn', knn)],
    voting = 'hard')
# hard -> voting, soft -> averaging
# soft 방법이 일반적인 보팅방법

In [ ]:
from sklearn.metrics import accuracy_score
for clf in (logreg, tree, knn, voting) :
    clf.fit(X_train, y_train)
    print(clf.__class__.__name__, 
          accuracy_score(y_test, clf.predict(X_test)))

LogisticRegression 0.6759611841297138
DecisionTreeClassifier 0.804692298243459
KNeighborsClassifier 0.6567989190517135


#### 4. pipe

In [ ]:
# standard syntax
pipe_long = Pipeline([("scaler",  StandardScaler()), 
                      ("svm", SVC(C=100))])
#pipe_long = Pipeline([('pca', PCA(n_components=3))), 
 #                     (('univ_select', SelectKBest(k=10))])
# abbreviated syntax 이름을 넣지 않고 파이프 라인 만들기
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100)) 

In [ ]:
print("Pipeline steps:\n{}".format(pipe_short.steps))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipe = make_pipeline(StandardScaler(), PCA(n_components=2), 
                     StandardScaler())
print("Pipeline steps:\n{}".format(pipe.steps))

In [ ]:
# create feature union
features = []
features.append(('pca', PCA(n_components=3)))
features.append(('univ_select', SelectKBest(k=10)))
feature_union = FeatureUnion(features)

# create pipeline
estimators = []
estimators.append(('features', feature_union))
estimators.append(('scaler',  StandardScaler()))
estimators.append(("svm", SVC()))
pipe = Pipeline(estimators)

In [ ]:
pipe.fit(X_train, y_train).score(X_test, y_test)

### Averaging predictions

In [ ]:
averaging = VotingClassifier(
    estimators = [('logreg', logreg), ('tree', tree), ('knn', knn)],
    voting = 'soft')
averaging.fit(X_train, y_train)

In [ ]:
averaging.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
svm = SVC()
mlp = MLPClassifier(alpha=1)

### Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(base_estimator=KNeighborsClassifier(), random_state=0, n_estimators=200)
bagging.fit(X_train, y_train).score(X_test, y_test)

### Boosting
- AdaBoost(Adaptive Boosting)
- Gradient Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(n_estimators=200, random_state=0)
ada.fit(X_train, y_train).score(X_test, y_test)

In [74]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=200, random_state=0)
gbm.fit(X_train, y_train).score(X_test, y_test)

0.8437538385947673

#### stacking

In [77]:
from mlxtend.classifier import StackingClassifier
import numpy as np
np.random.seed(1671)

stacking = StackingClassifier(classifiers=[tree, ada, gbm, pipe], 
                              meta_classifier=logreg, # blender or meta-learner
                              use_probas=False,
                              average_probas=False)

for clf in (tree, ada, gbm, pipe, stacking) :
    clf.fit(X_train, y_train)
    print(clf.__class__.__name__, accuracy_score(
        y_test, clf.predict(X_test)))

DecisionTreeClassifier 0.8069033288293821
AdaBoostClassifier 0.8415428080088441
GradientBoostingClassifier 0.8437538385947673
Pipeline 0.8105883798059207
StackingClassifier 0.8060434836015231


In [115]:
# 파라미터 서치
def bestGBDTNextModel(model, isKfold, nfold, searchCV, Xtrain, ytrain, Xtest, ytest, nIter, scoring, errScore, verbose, nJobs):
    # GridSearchCV을 위해 파라미터 값을 제한함.
    grd_prams = {}
    classifier = XGBClassifier(random_state=0, objective='binary:logistic')
    cv = KFold(n_splits=nfold, shuffle=True, random_state=0)
    
    if model == 'LGBM':
        # 그래디언트 부스팅 결정 트리(GBDT)    
        grd_prams.update({'max_depth': [50, 100],
              'learning_rate' : [0.01, 0.05],
              'num_leaves': [150, 200],
              'n_estimators': [300, 400],
              'num_boost_round':[4000, 5000],
              'subsample': [0.5, 1],
              'reg_alpha': [0.01, 0.1],
              'reg_lambda': [0.01, 0.1],
              'min_data_in_leaf': [20, 30],
              'lambda_l1': [0.01, 0.1],
              'lambda_l2': [0.01, 0.1]
            })
        
        #grd_prams.update({'max_depth': [50, 75, 90, 100],
        #      'learning_rate' : [0.01, 0.05, 0.07, 0.1],
        #      'num_leaves': [300,600,900,1200],
        #      'n_estimators': [100, 300, 500, 900],
        #      'num_boost_round':[1000, 2000, 3000, 4000],
        #      'num_leaves': [30, 60, 120, 150, 200],
        #      'reg_alpha': [0.01, 0.1, 0.5, 0.7, 1.0],
        #      'min_data_in_leaf': [50, 100, 300, 800],
        #      'lambda_l1': [0, 0.1, 0.5, 1.0],
        #      'lambda_l2': [0, 0.01, 1.0]})
        
        classifier = LGBMClassifier(random_state=0, boosting_type='gbdt', objective='binary', metric='auc')
        
    elif model == 'XGB':
        grd_prams.update({'n_estimators': [300, 500],
            'learning_rate': [0.001, 0.01],
            'subsample': [0.5, 1],
            'max_depth': [5, 6],
            'colsample_bytree': [0.97, 1.24],
            'min_child_weight': [1, 2],
            'gamma': [0.001, 0.005],
            'nthread': [3, 4],
            'reg_lambda': [0.5, 1.0],
            'reg_alpha': [0.01, 0.1]
          })
        
        #grd_prams.update({'n_estimators': [300, 500, 700],
        #    'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09],
        #    'subsample': [0.5, 1],
        #    'max_depth': [4, 5, 6, 7, 8, 9, 10],
        #    'colsample_bytree': [0.52, 0.97, 1,55, 2.32, 3.46],
        #    'min_child_weight': [1, 2, 3, 4],
        #    'gamma': [0.001, 0.01, 0.1, 0, 1],
        #    'nthread': [3, 4, 5],
        #    'reg_lambda': [0.01, 0.1, 0.5, 0.7, 1.0],
        #    'reg_alpha': [0.01, 0.1, 0.5, 0.7, 1.0]
        #  })
    
    if isKfold == False:
        cv = StratifiedShuffleSplit(n_splits=nfold, test_size=0.2, random_state=0)
    
    grid_ = RandomizedSearchCV(classifier, param_distributions=grd_prams, n_iter=nIter, scoring=scoring, error_score=errScore, verbose=verbose, n_jobs=nJobs, cv=cv)

    # 속도 이슈
    if searchCV == 'GRID': 
        grid_ = GridSearchCV(classifier, param_grid=grd_prams, n_jobs=nJobs, scoring=scoring, verbose=verbose, cv=cv)
    
    grid_.fit(Xtrain, ytrain)
    score_ = grid_.score(Xtest, ytest)
    
    #best = {"best_param":grid_.best_params_, 
    #        "best_score":grid_.best_score_, 
    #        "best_estimator":grid_.best_estimator_,
    #        "test_score":score_
    #       }
    
    print("{} grid_.best_score {}".format(model, np.round(grid_.best_score_,3)))
    print("{} grid_.best_score {}".format(model, np.round(score_,3)))
    print("{} best_estimator {}".format(model, grid_.best_estimator_))

    return grid_.best_params_

In [116]:
# scoring="roc_auc|f1" => 0.755
# categories_indices = [X_train.columns.get_loc(col) for col in ['주구매코너']]
best_param1 = bestGBDTNextModel('XGB', False, 5, 'RANDOM', X_train, y_train, X_test, y_test, 15, 'roc_auc', 0, 3, -1)
xgb = XGBClassifier(**best_param1)
score_xgb = xgb.fit(X_train, y_train).score(X_test, y_test)
print("score_lgbm1 ::: {}".format(score_xgb))
print("-----------------------------------")
y_xgb = xgb.predict(X_test)
print(classification_report(y_test, y_xgb))

# 최고=0.755 Score LGBM best_estimator
#  LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
#         importance_type='split', lambda_l1=0.01, lambda_l2=0,
#         learning_rate=0.01, max_depth=50, metric='auc',
#         min_child_samples=20, min_child_weight=0.001, min_data_in_leaf=20,
#         min_split_gain=0.0, n_estimators=300, n_jobs=-1,
#         num_boost_round=4000, num_leaves=150, objective='binary',
#         random_state=0, reg_alpha=0.1, reg_lambda=0.0, silent=True,
#         subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   32.4s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  3.8min finished


XGB grid_.best_score 0.932
XGB grid_.best_score 0.928
XGB best_estimator XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.97, gamma=0.001, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.01, max_delta_step=0, max_depth=6,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              n_estimators=300, n_jobs=4, nthread=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0.01,
              reg_lambda=1.0, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)
score_lgbm1 ::: 0.8374892519346517
-----------------------------------
              precision    recall  f1-score   support

           0       0.74      0.78      0.76      2701
           1       0.89      0.87      0.88      5440

    accuracy                           0.84      814

#### lightGBM

In [117]:
def bestGBDTNextModel(model, isKfold, nfold, searchCV, Xtrain, ytrain, Xtest, ytest, nIter, scoring, errScore, verbose, nJobs):
    grd_prams = {}
    classifier = LGBMClassifier(random_state=0, objective='binary:logistic')
    cv = KFold(n_splits=nfold, shuffle=True, random_state=0)
    if model == 'LGBM':  
        grd_prams.update({'max_depth': [50, 100],
              'learning_rate' : [0.01, 0.05],
              'num_leaves': [150, 200],
              'n_estimators': [300, 400],
              'num_boost_round':[4000, 5000],
              'subsample': [0.5, 1],
              'reg_alpha': [0.01, 0.1],
              'reg_lambda': [0.01, 0.1],
              'min_data_in_leaf': [20, 30],
              'lambda_l1': [0.01, 0.1],
              'lambda_l2': [0.01, 0.1]
            })
        classifier = LGBMClassifier(random_state=0, boosting_type='gbdt', objective='binary', metric='auc')
    if isKfold == False:
        cv = StratifiedShuffleSplit(n_splits=nfold, test_size=0.2, random_state=0)
    grid_ = RandomizedSearchCV(classifier, param_distributions=grd_prams,
                               n_iter=nIter, scoring=scoring, error_score=errScore, verbose=verbose, n_jobs=nJobs, cv=cv)
    grid_.fit(Xtrain, ytrain)
    score_ = grid_.score(Xtest, ytest)
    print("{} grid_.best_score {}".format(model, np.round(grid_.best_score_,3)))
    print("{} grid_.best_score {}".format(model, np.round(score_,3)))
    print("{} best_estimator {}".format(model, grid_.best_estimator_))
    return grid_.best_params_

In [ ]:
best_param = bestGBDTNextModel('LGBM', False, 5, 'RANDOM', X_train, y_train, X_test, y_test, 15, 'roc_auc', 0, 3, -1)
lgbm = LGBMClassifier(**best_param)
score_lgbm = lgbm.fit(X_train, y_train).score(X_test, y_test)
print("score_lgbm ::: {}".format(score_lgbm))
print("-----------------------------------")
y_lgbm = lgbm.predict(X_test)
print(classification_report(y_test, y_lgbm))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  8.5min


##### PR curve

In [ ]:
from sklearn.metrics import precision_recall_curve

def plot_precision_recall_curve(precisions, recalls) :
    plt.plot(recalls, precisions, color='blue')
    plt.axis([0,1,0,1])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PR curve')

In [ ]:
precisions, recalls, _ = precision_recall_curve(y_test, 
                                    gbm.predict_proba(X_test)[:,1])
plot_precision_recall_curve(precisions, recalls)

#### AUC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

In [ ]:
# 모델이 예측한 확률이 필요
# 0, 1 중 1의 값만 뽑아라
fpr, tpr, _ = roc_curve(y_test, gbm.predict_proba(X_test)[:,1])
auc(fpr, tpr)

In [ ]:
def plot_roc_curve(fpr, tpr, model, color=None) :
    model = model + ' (auc = %0.3f)' % auc(fpr, tpr)
    plt.plot(fpr, tpr, label=model, color=color)
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.axis([0,1,0,1])
    plt.xlabel('FPR (1 - specificity)')
    plt.ylabel('TPR (recall)')
    plt.title('ROC curve')
    plt.legend(loc="lower right")

In [ ]:
fpr_dummy, tpr_dummy, _ = roc_curve(y_test, 
                                    gbm.predict_proba(X_test)[:,1])
plot_roc_curve(fpr_dummy, tpr_dummy, 'gbm', 'hotpink')
fpr_tree, tpr_tree, _ = roc_curve(y_test, 
                                  ada.predict_proba(X_test)[:,1])
plot_roc_curve(fpr_tree, tpr_tree, 'ada', 'darkgreen')

In [ ]:
human_GBM = humanew.copy()
human_GBM.head()

In [ ]:
new = pd.read_csv('human_new.csv',encoding='cp949')
new.head()

In [ ]:
obj = ['노동 계급', '학력', '혼인 상태', '직업', '관계', '인종', '모국']
new[obj] = new[obj].apply(lambda x: x.astype('category').cat.codes)

In [ ]:
new = new.rename(columns={'아이디':'id', '나이':'age', '노동 계급':'workclass', '학력':'education', 
              '교육 수':'education_num', '혼인 상태':'marital_status', '직업':'occupation',
              '관계':'relationship', '인종':'race', '자본 이득':'capital_gain',
              '자본 손실':'capital_loss', '주당 시간':'hours_per_week', '모국':'native_country'})
new

In [ ]:
new.head()

In [ ]:
new['SEX'] =gbm.predict(new.loc[:,'age':'native_country'])

In [ ]:
new.head()

In [ ]:
new.head()

In [ ]:
str = ['id','SEX']
new[str].head()

In [ ]:
new[str].to_csv('kaggle2__data.csv',index=False)